In [212]:
import os
import tweepy
from dotenv import load_dotenv

import json

In [165]:
load_dotenv()

consumer_key = os.environ["API_KEY"]
consumer_secret = os.environ["API_KEY_SECRET"]
access_token = os.environ["ACCESS_TOKEN"]
access_token_secret = os.environ["ACCESS_TOKEN_SECRET"]
bearer_token = os.environ["BEARER_TOKEN"]


client = tweepy.Client(
  bearer_token=bearer_token,
  consumer_key=consumer_key,
  consumer_secret=consumer_secret,
  access_token=access_token,
  access_token_secret=access_token_secret,
  wait_on_rate_limit=True
)

In [166]:
def extraer_cuentas_usuarios(lista_cuentas, lista_tweets, response):
    #for i in range(len(response.data)):
    for i in range(len(response.includes['users'])):
        #lista_cuentas.append(response.includes['users'][i].username)
        lista_cuentas.append(response.includes['users'][i])
        lista_tweets.append(response.data[i].text)

    print(len(lista_cuentas))

In [192]:
# Search Recent Tweets
# This endpoint/method returns Tweets from the last seven days

#query = 'depresión yo tengo'
#query = 'Petro'
query = 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él'
#query = 'eafit'

# The method returns a Response object, a named tuple with data, includes,
# errors, and meta fields

cuentas_usuarios = []
tweets = []
for response in tweepy.Paginator(client.search_recent_tweets,
                                query,
                                tweet_fields=['id', 'author_id', 'text'],
                                user_fields=['id', 'username', 'location'],
                                expansions='author_id',
                                max_results=100, limit=1):#.flatten(limit=100):
    extraer_cuentas_usuarios(cuentas_usuarios, tweets, response)



100


In [193]:
print(len(cuentas_usuarios))

100


In [194]:
print(tweets)

['RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a ent…', 'RT @FisicoImpuro: Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van 

In [195]:
for usuario in cuentas_usuarios:
    print(usuario.location)

coloмвιa 
None
None
None
None
None
None
None
None
Maracay, venezuela
Bucaramanga, Colombia
None
None
None
CALI
None
Riohacha, Colombia
None
None
None
Cali, Valle del Cauca
Colombia
None
None
None
None
None
Cali, Colombia
Arauca
None
None
Iowa City, IA
None
None
None
Valledupar-cesar
CAJÚRU
None
Barranquilla colombia
None
None
None
None
None
Turquía
España
None
Pereira, Colombia
colombia-Tolima-Libano
Caloto, Colombia
None
Bogotá - Colombia 
Los Ojos de Kim Taehyung 
None
None
None
None
None
None
None
None
Putumayo, Colombia
Pasto, Nariño, Colombia
La Guajira,
None
None
Cali, Colombia
None
Puerto Berrio
Bogotá, D.C., Colombia
Bogotá, D.C., Colombia
Cali, Colombia
Bogotá
Medellin
Wageningen + Nederland
Colombia
BOGOTA D.C. COLOMBIANO 
None
None
Medellín
None
None
None
Santa Rosa de Osos, Colombia
Entre Bogotá Yopal Tolima
None
Por ahí/everywhere
None
Colombia
None
Val Müstair, Suiza
None
None
None
Colombia
None
La Virginia, Colombia
Bogotá
None
None


In [196]:
usuarios = []
ids_usuarios = []
for usuario in cuentas_usuarios:
    if (usuario.location is not None):
        if ('Medellin' in usuario.location) or ('Medellín' in usuario.location) or ('medellin' in usuario.location) or ('medellín' in usuario.location):
            usuarios.append(usuario.username)
            ids_usuarios.append(usuario.id)

In [197]:
print(usuarios)
print(ids_usuarios)

['Charlie4Gomez', 'carturo82']
[193148445, 1252978644]


In [198]:
diccionario_ids_usuarios = dict(zip(ids_usuarios, usuarios))

In [208]:
def extraer_tweets_usuarios(id, diccionario, response):
    #for i in range(len(response.data)):
    if id in list(diccionario.keys()):
        lista_tweets = diccionario[id]
    else:
        lista_tweets = []
    cont = 0
    for i in range(len(response.data)):
        if response.data[i]['referenced_tweets']:
            try:
                #print(cont, len(response.includes['tweets']))
                lista_tweets.append(response.includes['tweets'][cont].text)
                #print(response.data[i]['referenced_tweets'][0]['type'])
                cont += 1

                # print(cont)
                # print(response.includes['tweets'][cont])
                # #print(response.data[i]['referenced_tweets'][0]['type'])
                # print(response.data[i]['referenced_tweets'])
                # print()
            except:
                #print('Sobrepasó: ', cont)
                lista_tweets.append(response.data[i].text)

        else:
            lista_tweets.append(response.data[i].text)

    diccionario[id] = lista_tweets

In [209]:
users_tweets = dict()
for id_usuario in ids_usuarios:
    for response in tweepy.Paginator(client.get_users_tweets,
                                    id_usuario,
                                    tweet_fields=['id', 'author_id', 'text'],
                                    user_fields=['id', 'username', 'location'],
                                    expansions='referenced_tweets.id',
                                    #expansions='author_id',
                                    max_results=100, limit=1):#.flatten(limit=100):
        extraer_tweets_usuarios(id_usuario, users_tweets, response)
        # print(len(response.includes['tweets']))
        # print(response.includes['tweets'])
        # print(response.includes['tweets'][85])
        # break
        #print(response.includes['tweets'])

    #     print(len(response.includes))
    #     for i in range(len(response.data)):
    #         if response.data[i]['referenced_tweets']:
    #             print(response.data[i]['referenced_tweets'][0]['type'])
    #         else:
    #             print(response.data[i])
    #     break

In [210]:
print(users_tweets.keys())

dict_keys([193148445, 1252978644])


In [211]:
print(list(users_tweets.keys())[0])
print(users_tweets[list(users_tweets.keys())[0]])

193148445
['Gobierno Biden anuncia visita a Colombia para reunión con presidente Gustavo Petro https://t.co/VN0lPwFjkR', 'Se asustan por el "No nos reten" pero se hicieron los maricas con el "Se callan o los callamos"...', 'Ecopetrol debe tener un guardado ni el hijueputa.', 'La gran diferencia de la reforma tributaria del gobierno de @petrogustavo  es que no generará  protestas masivas, porque todos sabemos que es por el bien del país y no para que se beneficien unos cuantos a costa del sacrificio de otros!', 'El euro cayó al nivel más bajo de los últimos 20 años y se convirtió en 1 dólar.\n\n¿Alguien sabe por qué no han dicho en Alemania que es culpa de Gustavo Petro?', 'ALGUIEN SABE Q HACE LA MINISTRA CORCHO EN VENEZUELA?', 'Saben? A veces me pongo en los zapatos de Gustavo Petro y hasta me compadezco de él. El mierdero de país que le van a entregar y lo que va a tener que hacer arreglarlo; eso no se lo deseo ni a mi peor enemigo. Pero si hay alguien que puede lograrlo, es justament

In [204]:
print(diccionario_ids_usuarios[193148445])

Charlie4Gomez


In [158]:
print(len(users_tweets[list(users_tweets.keys())[0]]))

200


In [213]:
with open("tweets.json", "w") as fp:
    json.dump(users_tweets,fp)

In [214]:
with open("users.json", "w") as fp:
    json.dump(diccionario_ids_usuarios,fp)

In [ ]:


# #print(tweets)

# # Each Tweet object has default ID and text fields
# for tweet in tweets:
#     print(tweet.id)
#     print(tweet)
#     print(tweet.text)